In [1]:
from base58 import b58decode, b58encode

from solanaweb3.message import *
from solanaweb3.blockhash import Blockhash
from solanaweb3.publickey import PublicKey
from solanaweb3.transaction import Transaction, SIG_LENGTH

def print_message(msg):
    print("Header: ", msg.header)
    print("Recent Blockhash: ", b58decode(msg.recent_blockhash).hex())
    any((print(f"key{i + 1}: {key.to_buffer().hex()}") for i, key in enumerate(msg.account_keys)))
    print("Instructions: ", msg.instructions)

In [2]:
message_header = MessageHeader(0, 3, 2)
accounts_keys = [str(PublicKey(i)) for i in range(1, 6)]
recent_blockhash = Blockhash(str(PublicKey(1)))
instructions = [CompiledInstruction(accounts=[1,2,3], data=b58encode(bytes([9] * 5)), program_id_index=4)]
args = MessageArgs(message_header, accounts_keys, recent_blockhash, instructions)
msg = Message(args)
msg_bytes = msg.serialize()
msg_bytes

b'\x00\x03\x02\x05\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x04\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x05\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x01\x04\x03\x01\x02\x03\x05\t\t\t\t\t'

In [3]:
decoded_msg = msg.deserialize(msg_bytes)
assert decoded_msg.header == message_header
assert decoded_msg.recent_blockhash == recent_blockhash
print_message(decoded_msg)

Header:  MessageHeader(num_required_signatures=0, num_readonly_signed_accounts=3, num_readonly_unsigned_accounts=2)
Recent Blockhash:  0000000000000000000000000000000000000000000000000000000000000001
key1: 0000000000000000000000000000000000000000000000000000000000000001
key2: 0000000000000000000000000000000000000000000000000000000000000002
key3: 0000000000000000000000000000000000000000000000000000000000000003
key4: 0000000000000000000000000000000000000000000000000000000000000004
key5: 0000000000000000000000000000000000000000000000000000000000000005
Instructions:  [CompiledInstruction(accounts=b'\x01\x02\x03', program_id_index=4, data=b'228BPqr')]


In [4]:
signatures = [
    b58encode(bytes([1] * SIG_LENGTH)),
    b58encode(bytes([2] * SIG_LENGTH)),
];
# Note: Signatures will be added to transaction on each run
tx = Transaction.populate(msg, signatures)
tx_msg = tx.compile_message()
assert tx.serialize_message() == tx_msg.serialize()
assert len(tx_msg.instructions) == len(tx.instructions) == 1
assert tx_msg.recent_blockhash == tx.recent_blockhash == recent_blockhash
print_message(tx_msg)

Header:  MessageHeader(num_required_signatures=2, num_readonly_signed_accounts=0, num_readonly_unsigned_accounts=2)
Recent Blockhash:  0000000000000000000000000000000000000000000000000000000000000001
key1: 0000000000000000000000000000000000000000000000000000000000000001
key2: 0000000000000000000000000000000000000000000000000000000000000002
key3: 0000000000000000000000000000000000000000000000000000000000000003
key4: 0000000000000000000000000000000000000000000000000000000000000004
key5: 0000000000000000000000000000000000000000000000000000000000000005
Instructions:  [CompiledInstruction(accounts=[1, 2, 3], program_id_index=4, data=b'228BPqr')]
